![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/annotation/english/spark-nlp-basics/spark-nlp-basics-functions.ipynb)

## 0. Colab Setup

In [ ]:
# This is only to setup PySpark and Spark NLP on Colab
!wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

openjdk version "1.8.0_252"
OpenJDK Runtime Environment (build 1.8.0_252-8u252-b09-1~18.04-b09)
OpenJDK 64-Bit Server VM (build 25.252-b09, mixed mode)
     |████████████████████████████████| 215.7MB 60kB/s 
     |████████████████████████████████| 204kB 48.0MB/s 
     |████████████████████████████████| 122kB 2.8MB/s 


In [ ]:
from sparknlp.annotator import *

In [ ]:
import sparknlp 

spark = sparknlp.start()

print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)


Spark NLP version:  2.5.0
Apache Spark version:  2.4.4


In [ ]:
from sparknlp.pretrained import *

In [ ]:
data = spark.createDataFrame([['Peter is a goud person.']]).toDF('text')

In [ ]:
pipeline = PretrainedPipeline('explain_document_ml')

explain_document_ml download started this may take some time.
Approx size to download 9.4 MB
[OK!]


In [ ]:
result = pipeline.transform(data)

In [ ]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|               spell|              lemmas|               stems|                 pos|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|Peter is a goud p...|[[document, 0, 22...|[[document, 0, 22...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[token, 0, 4, Pe...|[[token, 0, 4, pe...|[[pos, 0, 4, NNP,...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+



In [ ]:
from sparknlp.functions import *

In [ ]:
from sparknlp.annotation import Annotation

def my_annoation_map_function(annotations):
    return list(map(lambda a: Annotation(
        'my_own_type',
        a.begin,
        a.end,
        a.result,
        {'my_key': 'custom_annotation_data'},
        []), annotations))

In [ ]:
# The array type must be provided in order to tell Spark the expected output type of our column.
# We are using an Annotation array here

result.select(
    map_annotations(my_annoation_map_function, Annotation.arrayType())('token')
).toDF("my output").show(truncate=False)

+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|my output                                                                                                                                                                                                                                                                                                                                                                                                          |
+-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

In [ ]:
# we can also explode annotations like this

explode_annotations_col(result, 'lemma.result', 'exploded').select('exploded').show()

AnalysisException: ignored